In [2]:
import pandas as pd, math, numpy as np
import cirq
import mat2qubit as m2q
from openfermion import QubitOperator
from itertools import combinations
import import_ipynb
import Hermite_helper_fns as Helper

# import Hermite_rep.ipynb

importing Jupyter notebook from Hermite_helper_fns.ipynb


In [224]:
def str_Hermite_encoding_X_or_P(op,d,coeff=1):
    '''
    Return the Hermite representation for a position or momentum operator

    Input:
    op: operator to be encoded, e.g. "X2", "P1", "X1_0 X2_1"
    d: list of d-levels
    coeff: numerical coefficient of the operator term
    
    Yield:
    Hermite_op: The Hermite representation of the operator
    '''
    
    # Creating an list of qubit indices
    temp_qub_counter=0
    tot_qubits=[]
    
    if type(d)==int:
        assert Helper.is_power_of_two(d)
        num_q = int(math.log(d,2))
        for counter in range(num_q):
            temp_list.append(counter)
    elif type(d)==list:
        for register in d:
            assert Helper.is_power_of_two(register)
            num_q = int(math.log(register,2)) # number of qubits required for the register
            temp_list=[]
            if not tot_qubits:
                for counter in range(num_q):
                    temp_list.append(counter)
                temp_qub_counter=temp_list[-1]
            else:
                for counter in range(num_q):
                    t_ind = temp_qub_counter+counter+1
                    temp_list.append(t_ind)
                temp_qub_counter=temp_list[-1]
            
            tot_qubits.append(temp_list)
    
    
    
    terms = op.split(" ")
    Hermite_op=QubitOperator()
    powers=[]
    
    # Create list of powers of given operators
    for term in terms:
        if len(term)==1:
            powers.append(1)
        elif Helper.is_number(term[1]):
            powers.append(int(term[1]))
        else:
            powers.append(1)
    
    if len(terms)==1:
        # Only one operator
        power=powers[0]
        if len(terms[0])==1:
            # Only a single register
            nq = num_q # number of qubits
            for i in range(nq):
                Hermite_op += pow(2,i)*QubitOperator(f"Z{i}")
            if power>1:
                for expo in range(power-1):
                    Hermite_op *= Hermite_op
                    
        if len(terms[0])==2: 
            # Only a single register
            nq=int(math.log(d[0],2))
            for i in range(nq):
                Hermite_op += pow(2,i)*QubitOperator(f"Z{i}")
                # Hermite_dict[f"Z{i}"]=pow(2,i)
            if power>1:
                for expo in range(power-1):
                    Hermite_op *= Hermite_op
                    
        elif len(terms[0])==4 or len(terms[0])==3: 
            # Multiple registers
            if len(terms[0])==3:
                register_ind = int(terms[0][2])
            elif len(terms[0])==4:
                register_ind = int(terms[0][3])
                
            nq=int(math.log(d[register_ind],2)) # number of qubits in the register
            temp_dict={}
            temp_Hermite_op=QubitOperator()
            for j in range(nq):
                qubit_number=tot_qubits[register_ind][j]
                temp_Hermite_op += pow(2,j)*QubitOperator(f"Z{qubit_number}")
            if power>1:
                for expo in range(power-1):
                    temp_Hermite_op *= temp_Hermite_op
                    
            Hermite_op += temp_Hermite_op
        
        Hermite_op *= coeff
            
    elif len(terms)>=2: 
        # Two operators 
        
        # Parse through each term in the given operator
        for i in range(len(terms)):
            term=terms[i]
            if len(term)==3:
                register_ind = int(term[2])
            elif len(term)==4:
                register_ind = int(term[3])
            power=powers[i]
            nq=int(math.log(d[register_ind],2)) 
            temp_dict={}
            temp_Hermite_op=QubitOperator()
            # Conversion into Z operators
            for j in range(nq):
                qubit_number=tot_qubits[register_ind][j]
                temp_Hermite_op += pow(2,j)*QubitOperator(f"Z{qubit_number}")
                
            # Raising to a power>1
            if power>1:
                for expo in range(power-1):
                    temp_Hermite_op *= temp_Hermite_op
                    
            test = QubitOperator()
            if Hermite_op==test:
                Hermite_op += temp_Hermite_op
            else:
                Hermite_op *= temp_Hermite_op
            
        # Multiplying by numerical coefficient
        Hermite_op *= coeff

    return Hermite_op

In [252]:
def Hermite_encoding(ham,d):
    '''
    Given a qSymbOp, procure the Hermite encoding representation
    
    Args:
    ham: given Hamiltonian (as type qSymbOp)
    d: list of d-levels
    
    Yield:
    Enc_op: Hermite encoded representation of the given operator
    '''
    XP_list=[] # Contains separated X and P terms for XP operators
    PX_list=[] # Contains separated X and P terms for PX operators
    
    X_enc_op=QubitOperator()
    P_enc_op=QubitOperator()
    
    
    # Converting qSymbOp to string
    str_ham=str(ham)
    # Parsing through the terms in the string
    split_str_ham = str_ham.split("++")
    for term in split_str_ham:
        subterm_list=term.split()
        
        # Checking for a numerical coefficient
        if Helper.is_number(subterm_list[0].replace("(","").replace(")","")):
            coeff=complex(subterm_list[0].replace("(","").replace(")",""))
            # print(coeff)
            subterm_list.pop(0)
        else:
            coeff=1
        
        # Creating an operator string without numberical coefficient
        op_str=""
        for op in subterm_list:
            if op_str=="":
                op_str+=op.replace("[","").replace("]","")
            else:
                op_str+= " "+op.replace("[","").replace("]","")
        
        
        # Determining whether the op term is pure X, pure P, XP, or PX
        x_str=""
        p_str=""
        xp_str=""
        px_str=""
        
        counter0=0
        for c1_ind in range(len(op_str)):
            c=op_str[c1_ind].upper()
            if c=="X" or c=="Q":
                counter0+=1
                counter1=0
                for c2_ind in range(len(op_str)):
                    E=op_str[c2_ind].upper()
                    if E=="P" and c2_ind>c1_ind:
                        xp_str=op_str
                        counter1+=1
                    elif E=="P" and c2_ind<c1_ind:
                        px_str=op_str
                        counter1+=1
                if counter1==0:
                    x_str=op_str
                    
        if counter0==0:
            p_str=op_str
        
        
        # Converting the operator term into Hermite representation
        if x_str:
            X_enc_op += str_Hermite_encoding_X_or_P(x_str,d,coeff)
        elif p_str:
            P_enc_op += str_Hermite_encoding_X_or_P(p_str,d,coeff)
        elif xp_str or px_str:
            mix_list_X_enc=QubitOperator()
            mix_list_P_enc=QubitOperator()
            if xp_str:
                x_part = xp_str.split()[0]
                p_part = xp_str.split()[1]
                
                mix_list_X_enc += str_Hermite_encoding_X_or_P(x_part,d,coeff)
                
                mix_list_P_enc += str_Hermite_encoding_X_or_P(p_part,d,1)
                
                sep_list=[mix_list_X_enc,mix_list_P_enc]
                XP_list.append(sep_list)
            
            elif px_str:
                x_part = px_str.split()[1]
                p_part = px_str.split()[0]
                
                mix_list_X_enc += str_Hermite_encoding_X_or_P(x_part,d,coeff)
                
                mix_list_P_enc += str_Hermite_encoding_X_or_P(p_part,d,1)
                
                sep_list=[mix_list_X_enc,mix_list_P_enc]
                PX_list.append(sep_list)
            
            
    return X_enc_op, P_enc_op, XP_list, PX_list